<div style="background-color:#FF0000">Отсутствуют запросы с update!!!</div>

ElasticSearch - это документарная база данных с возможностью полнотекстовго и географического поиска.

Попробуем решить с ее помощью знакомую задачу - выкачать NPlus1 и положить статьи в базу данных.

База данных [ставится](https://www.elastic.co/guide/en/elasticsearch/reference/current/install-elasticsearch.html) довольно просто, дальше к ней можно обращаться просто по локальному адресу, порт 9200: https://127.0.0.1:9200/ . На все запросы база данных будет присылать JSON-документ. То есть при желании с ней можно работать при помощи библиотеки `requests`. Вместо этого поставим библиотеку `elasticsearch` и будем работать через нее.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import datetime
from tqdm.auto import tqdm

# Импортируем библиотеку для БД.
from elasticsearch import Elasticsearch


К базе данных нужно обращаться при помощи класса `Elasticsearch`. 

In [2]:
# Подключаемся к базе данных.
es=Elasticsearch()

In [3]:
es

<Elasticsearch([{}])>

Создадим класс `NPlus1Article`, который будет хранить информацию по статье и преобразовывать ее к словарю или JSON. Используем старые функции, которые выгружают статьи за весь день и статьи за заданный период.

In [5]:
delcom=re.compile("<!--.+-->", re.S)

# Класс, хранящий информацию о статье.
class NPlus1Article:
    def __init__(self):
        self.time = ""
        self.date = ""
        self.rubr = ""
        self.diff = ""
        self.author = ""
        self.head = ""
        self.text = ""
        
    # Конвертация в JSON.
    def toJSON(self):
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "rubrics":"'+self.rubr+ \
            '", "difficulty":"'+self.diff+'", "title":"'+self.head+'", "author":"'+ \
            self.author+'","text":"'+self.text.replace('"', '\\"')+'"}'
        return res

    # Конвертация в словарь.
    def toDict(self):
        res={"date":self.date, "time":self.time, "rubrics":self.rubr, "difficulty":self.diff,\
             "title":self.head, "author":self.author,"text":self.text.replace('"', '\\"')}
        return res
    
def getArticleTextNPlus1(adr):
    r = requests.get(adr)
    #print(r.text)
    art = NPlus1Article()
    tables = re.split("</div>", re.split('="tables"', r.text)[1])[0]
    t1 = re.split("</time>", re.split("<time", tables)[1])[0]
    art.time = re.split("</span>", re.split("<span>", t1)[1])[0]
    art.date = re.split("</span>", re.split("<span>", t1)[2])[0]
    art.rubr = re.findall("<a data-rubric.+?>(.+?)</a>", r.text)[0]
    art.diff = re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
    art.head = re.split("</h1>",re.split('<h1>', r.text)[1])[0]
    art.author = re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
    art.text = re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

    beaux_text = BeautifulSoup(art.text, "html5lib")
    art.text = delcom.sub("", beaux_text.get_text() )
    art.text = art.text.replace('\xa0', ' ')

    # print(art.n_time, art.n_date, art.n_rubr, art.n_diff)
    # print(art.n_head)
    # print(art.n_author)
    # print(art.n_text)
    #return [n_time, n_date, n_rubr, n_diff, n_author, n_head, n_text]
    return art

def getDayArticles(adr):
    r = requests.get(adr)
    titles = BeautifulSoup(r.text, "html5lib")("article")
    #print(titles)
    addrs = ["https://nplus1.ru/"+a("a")[0]["href"] for a in titles]
    #print(addrs)
    articles = []
    for adr in addrs:
        articles.append(getArticleTextNPlus1(adr))
    return articles

Загрузим статьи за несколько дней.

In [38]:
arts=getDayArticles("https://nplus1.ru/news/2020/11/14/")
#arts=getDayArticles("https://nplus1.ru/news/2020/10/19")
#arts=getDayArticles("https://nplus1.ru/news/2017/10/16/")
#arts=getDayArticles("https://nplus1.ru/news/2016/08/04/")

In [31]:
arts[0].head

'У атомного лихтеровоза «Севморпуть» сломался винт по пути в Антарктиду. Ему снимут лопасть'

Вся информация хранится в так называемых индексах (аналог базы данных). 

Создадим индекс при помощи `indices.create(index=название_индекса)`

In [8]:
es.indices.create(index="nplus1")

RequestError: RequestError(400, 'resource_already_exists_exception', 'index [nplus1/gx1_sJVORUqcYBW_8Bas-g] already exists')

Для индекса можно задать тип документа, который будет в нем храниться. Если тип не будет задан, БД сама создаст его при первой загрузке документа (не всегда так, как этого хотелось бы нам, поэтому лучше тип задавать явно).

Описание типа (маппинг) ведется в виде JSON или словаря. Оно должно содержать в себе название типа (в нашем случае это "article") и описание полей. Описание полей дается после ключевого слова "properties". Для каждого поля задается его название за которым следует описание поля: тип, формат, анализатор (язык), другие параметры.

Маппинг для существующего индекса создается при помощи `indices.put_mapping`, в [который](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.client.IndicesClient.put_mapping) передается названия индекса и типа. Можно передать маппинг сразу в функцию создания индекса.

In [10]:
mapit={"article":{"properties":{"author":{"type":"text"},
                                "date":{"type":"text"},
                                "time":{"type":"date", "format":"HH:mm"},
                                "difficulty":{"type":"double"},
                                "rubrics":{"type":"text","analyzer" : "russian"},
                                "text":{"type":"text","analyzer" : "russian"},
                                "title":{"type":"text","analyzer" : "russian"}}}}

es.indices.put_mapping(index="nplus1", doc_type='article', body=mapit, include_type_name=True)

{'acknowledged': True}

При желании индекс можно [удалить](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.client.IndicesClient.delete).

In [4]:
es.indices.delete("nplus1")

{'acknowledged': True}

Можно получить список индексов для текущей базы.

In [42]:
es.indices.get_alias("*")

{'syntagrus_rel': {'aliases': {}},
 'cosyco': {'aliases': {}},
 'nplus1': {'aliases': {}},
 'syntagrus': {'aliases': {}}}

Можно получить маппинг для интересующего нас индекса.

In [43]:
es.indices.get_mapping(index="nplus1", doc_type='article', include_type_name=True)

{'nplus1': {'mappings': {'article': {'properties': {'author': {'type': 'text'},
     'date': {'type': 'text'},
     'difficulty': {'type': 'double'},
     'rubrics': {'type': 'text', 'analyzer': 'russian'},
     'text': {'type': 'text', 'analyzer': 'russian'},
     'time': {'type': 'date', 'format': 'HH:mm'},
     'title': {'type': 'text', 'analyzer': 'russian'}}}}}}

Загрузка документов в базу ведется при помощи функции `index`. Тем самым БД и библиотека подчеркивают, что для них строится полнотекстовый индекс для быстрого поиска. [В функцию](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.Elasticsearch.index) передаются название индекса и типа документа. Также можно передать идентификатор документа и другие параметры.

In [39]:
%%time

for art in tqdm(arts):
    es.index(index="nplus1", doc_type='article', body=art.toDict())


CPU times: user 81.4 ms, sys: 0 ns, total: 81.4 ms
Wall time: 1.18 s


Получить документ по его идентификатору можно при помощи [функции](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.Elasticsearch.get) `get`.

Но в дальнейшем мы будем пользоваться [функцией](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.Elasticsearch.search) `search`, которая позволяет проводить любой поиск, фильтрацию и аггрегацию информации. Для поиска надо передать название индекса и запрос в виде JSON или словаря.

В простейшем случае запрос содержит в себе поле `query`, включающее в себе описание документов, которые мы хотим найти. Также в запрос могут включаться другие параметры: количество выводимых документов, фильтры, аггрегация и др.

В запросе ниже мы используем `match_all`, то есть просьбу вывести все документы. По умолчанию выводятся 10 документов.

Результат содержит в себе несколько полей, `["hits"]["hits"]` содержит найденные документы.

In [45]:
#es.get(index="nplus1", doc_type='article', id=1)
res = es.search(index="nplus1", body={"query": {"match_all": {}}})
res["hits"]["hits"]
#res

[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'G-W2FHABtPFlvM7fhAWd',
  '_score': 1.0,
  '_source': {'date': '01 Фев. 2019',
   'time': '20:45',
   'rubrics': 'Космонавтика',
   'difficulty': '2.3',
   'title': 'На обратной стороне Луны зафиксирована рекордно низкая ночная температура',
   'author': 'Александр Войтюк',
   'text': 'Ночная температура на поверхности обратной стороны Луны оказалась ниже минимальных значений, зафиксированных ночью на видимой стороне. Такие данные были получены после того, как 29 января луноход «Юйту-2» и посадочная платформа «Чанъэ-4» успешно вышли из режима гибернации. Об этом 1 февраля сообщается на сайте Китайского национального космического агентства.\n\nАвтоматическая станция «Чанъэ-4» с луноходом «Юйту-2» совершила мягкую посадку на обратной стороне Луны 3 января 2019 года. Связь между посадочной платформой и Землей поддерживается при помощи спутника-ретранслятора «Цюэцяо», находящегося на гало-орбите вокруг точки Лагранжа L2 системы Земля-Лун

Сам по себе запрос содержит информацию о шардах, найденные документы, их количество, информацию о выполнении запроса.

In [13]:
len(res["hits"]["hits"])

10

In [14]:
res

{'took': 754,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 40, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'nplus1',
    '_type': 'article',
    '_id': 'G-W2FHABtPFlvM7fhAWd',
    '_score': 1.0,
    '_source': {'date': '01 Фев. 2019',
     'time': '20:45',
     'rubrics': 'Космонавтика',
     'difficulty': '2.3',
     'title': 'На обратной стороне Луны зафиксирована рекордно низкая ночная температура',
     'author': 'Александр Войтюк',
     'text': 'Ночная температура на поверхности обратной стороны Луны оказалась ниже минимальных значений, зафиксированных ночью на видимой стороне. Такие данные были получены после того, как 29 января луноход «Юйту-2» и посадочная платформа «Чанъэ-4» успешно вышли из режима гибернации. Об этом 1 февраля сообщается на сайте Китайского национального космического агентства.\n\nАвтоматическая станция «Чанъэ-4» с луноходом «Юйту-2» совершила мягкую посадку на обратно

В качестве описания документов можно передать более конкретное описание, задаваемое ключевым словом `match`. Ниже мы просим найти все документы, поле `title` которых содержит слово `пустота`.

Если теперь вернуться к описанию типа (маппингу), то мы увидим, что поле `title` задавалось типом `text` с русским анализатором. Это означает, что все слова в документе будут приведены к начальной форме и поиск по ним будет вестись вне зависимости от формы слова. Если мы забудем задать русский язык, то по умолчанию возьмется английский, то есть нормального ндексирования и поиска не получится. Если бы хотели искать по точному совпадению значения поля, то следовало бы использовать тип `keyword`. Также можно использовать другие типы: `integer`, `float`, `date` и другие. Для всех полей можно задать форматирование, которое зависит от типа данных. В нашем случае мы задали, что время содержит в себе часы и минуты.

In [21]:
res = es.search(index="nplus1", body={"query": {"match": {"title": "пустота"}}})
res["hits"]["hits"]

[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'IuW2FHABtPFlvM7fhwXB',
  '_score': 3.1697235,
  '_source': {'date': '01 Фев. 2019',
   'time': '12:18',
   'rubrics': 'Экология и климат',
   'difficulty': '1.1',
   'title': 'Ученые нашли в антарктическом леднике пустоту размером с Подольск',
   'author': 'Ольга Добровидова',
   'text': 'Ученые из США, Германии и Франции нашли в антарктическом леднике Туэйтса полость площадью около 40 квадратных километров и глубиной 350 метров, которая сформировалась там после 2010 года. Размер полости таков, что в нее, по-видимому, целиком поместился бы подмосковный город Подольск. Статья об исследовании опубликована в журнале Science Advances.\nЛедник Туэйтса в Западной Антарктиде впадает в залив Пайн-Айленд моря Амундсена. Он известен высокой скоростью движения — до двух километров в год, и считается одним из «слабых мест» антарктического ледового щита с точки зрения потери льда и повышения уровня Мирового океана: если ледник растает полностью,

In [16]:
res

{'took': 182,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 3.1697235,
  'hits': [{'_index': 'nplus1',
    '_type': 'article',
    '_id': 'IuW2FHABtPFlvM7fhwXB',
    '_score': 3.1697235,
    '_source': {'date': '01 Фев. 2019',
     'time': '12:18',
     'rubrics': 'Экология и климат',
     'difficulty': '1.1',
     'title': 'Ученые нашли в антарктическом леднике пустоту размером с Подольск',
     'author': 'Ольга Добровидова',
     'text': 'Ученые из США, Германии и Франции нашли в антарктическом леднике Туэйтса полость площадью около 40 квадратных километров и глубиной 350 метров, которая сформировалась там после 2010 года. Размер полости таков, что в нее, по-видимому, целиком поместился бы подмосковный город Подольск. Статья об исследовании опубликована в журнале Science Advances.\nЛедник Туэйтса в Западной Антарктиде впадает в залив Пайн-Айленд моря Амундсена. Он известен в

Если мы хотим искать точное совпадение формы слова, следует использовать ключевое слово `term`.

In [27]:
res = es.search(index="nplus1", body={"query": {"term": {"title": "пустот"}}})
res["hits"]["hits"]

[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'IuW2FHABtPFlvM7fhwXB',
  '_score': 3.1697235,
  '_source': {'date': '01 Фев. 2019',
   'time': '12:18',
   'rubrics': 'Экология и климат',
   'difficulty': '1.1',
   'title': 'Ученые нашли в антарктическом леднике пустоту размером с Подольск',
   'author': 'Ольга Добровидова',
   'text': 'Ученые из США, Германии и Франции нашли в антарктическом леднике Туэйтса полость площадью около 40 квадратных километров и глубиной 350 метров, которая сформировалась там после 2010 года. Размер полости таков, что в нее, по-видимому, целиком поместился бы подмосковный город Подольск. Статья об исследовании опубликована в журнале Science Advances.\nЛедник Туэйтса в Западной Антарктиде впадает в залив Пайн-Айленд моря Амундсена. Он известен высокой скоростью движения — до двух километров в год, и считается одним из «слабых мест» антарктического ледового щита с точки зрения потери льда и повышения уровня Мирового океана: если ледник растает полностью,

БД ищет любые слова из запроса за исключением стоп-слов. Порядок слов при этом не важен. Наличие слов в итоговом документе в итоге влияет на поле `_score`, высчитываемое на основе меры tf*idf.

In [47]:
res = es.search(index="nplus1", body={"query": {"match": {"text": "Германии и Франции"}}})
res["hits"]["hits"]

[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'IuW2FHABtPFlvM7fhwXB',
  '_score': 6.7511806,
  '_source': {'date': '01 Фев. 2019',
   'time': '12:18',
   'rubrics': 'Экология и климат',
   'difficulty': '1.1',
   'title': 'Ученые нашли в антарктическом леднике пустоту размером с Подольск',
   'author': 'Ольга Добровидова',
   'text': 'Ученые из США, Германии и Франции нашли в антарктическом леднике Туэйтса полость площадью около 40 квадратных километров и глубиной 350 метров, которая сформировалась там после 2010 года. Размер полости таков, что в нее, по-видимому, целиком поместился бы подмосковный город Подольск. Статья об исследовании опубликована в журнале Science Advances.\nЛедник Туэйтса в Западной Антарктиде впадает в залив Пайн-Айленд моря Амундсена. Он известен высокой скоростью движения — до двух километров в год, и считается одним из «слабых мест» антарктического ледового щита с точки зрения потери льда и повышения уровня Мирового океана: если ледник растает полностью,

Если мы хотим найти все слова из запроса, к нему следует добавить параметр `operator`созначением `and`.

In [53]:
res = es.search(index="nplus1", body={"query": {"match": {"text": 
                                       {"query":"Франции Германии и", "operator": "and"}}}})
res["hits"]["hits"]

[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'IuW2FHABtPFlvM7fhwXB',
  '_score': 6.7511806,
  '_source': {'date': '01 Фев. 2019',
   'time': '12:18',
   'rubrics': 'Экология и климат',
   'difficulty': '1.1',
   'title': 'Ученые нашли в антарктическом леднике пустоту размером с Подольск',
   'author': 'Ольга Добровидова',
   'text': 'Ученые из США, Германии и Франции нашли в антарктическом леднике Туэйтса полость площадью около 40 квадратных километров и глубиной 350 метров, которая сформировалась там после 2010 года. Размер полости таков, что в нее, по-видимому, целиком поместился бы подмосковный город Подольск. Статья об исследовании опубликована в журнале Science Advances.\nЛедник Туэйтса в Западной Антарктиде впадает в залив Пайн-Айленд моря Амундсена. Он известен высокой скоростью движения — до двух километров в год, и считается одним из «слабых мест» антарктического ледового щита с точки зрения потери льда и повышения уровня Мирового океана: если ледник растает полностью,

Также можно задавать интервалы поиска при помощи ключевого слова `range`. `gte` и `lt` задают операторы "больше или равно" и "меньше".

In [57]:
res = es.search(index="nplus1", body={"query": {"range": {"time": 
                                       {"gte":"10:00", "lt": "12:00"}}}})
res["hits"]["hits"]


[{'_index': 'nplus1',
  '_type': 'article',
  '_id': 'JOW2FHABtPFlvM7fiAVK',
  '_score': 1.0,
  '_source': {'date': '01 Фев. 2019',
   'time': '11:24',
   'rubrics': 'Биология',
   'difficulty': '3.4',
   'title': 'Белок антибактериальной обороны у дрозофил взял на себя управление сном',
   'author': 'Александра Кочеткова',
   'text': 'Один из белков, отвечающий за защиту от бактериальных инфекций у дрозофил, способствует также засыпанию и крепкому сну, пишут ученые в статье, опубликованной в Science. Выработка этого белка усиливается всего в двух нейронах мозга мушек при долгом бодрствовании, а также при бактериальной инфекции. Результаты исследования \nСон регулируется циркадными ритмами и гомеостатической системой. Циркадные ритмы влияют на периоды сна и бодрствования в течении суток. Работа гомеостатической системы связана с накоплением факторов, вызывающих непреодолимое желание спать. Гомеостатические механизмы срабатывают при слишком длительном бодрствовании, или при дополнительн

---

Теперь немного отвлечемся на замеры производительности ElasticSearch. Скорость ее работы с отдельными запросами оставляет желать лучшего - несколько десятков запросов в секунду. Гораздо лучше она работает при помощи потоковых запросов, отправляющих на запись сразу много документов.

Заметим, что при параллельной отправке документов БД работает быстрее, но до миллиона запросов в секунду всё равно далеко. Для того, чтобы получить такую скорость необходим кластер.

Попробуем положить в базу первый миллион сочетаний "существительное+прилагательное" из базы [КОСИКО](http://cosyco.ru/). Для этого создадим индекс и зададим тип документов.

In [58]:
es.indices.delete("cosyco")
es.indices.create(index = "cosyco")
mapit = {"combinations" : {"properties" : {"noun" : {"type" : "text", "analyzer" : "russian"},
                                           "adj"  : {"type" : "text", "analyzer" : "russian"},
                                           "frq"  : {"type": "integer"}}}}

es.indices.put_mapping(index = "cosyco", 
                       doc_type = 'combinations', 
                       body = mapit, 
                       include_type_name=True)

{'acknowledged': True}

Код ниже обеспечивает около 10 запросов в секунду - просто слезы.

In [59]:
fil = open("noun_adj_inf_combs.txt")
cntr = 0
for line in tqdm(fil):
    r = line.split(";")
    r={"noun":r[0], "adj":r[1], "frq": r[2]}
    #r = {"noun":r[0], "adj":r[1]}
    es.index(index="cosyco", doc_type='combinations', body=r)
    cntr += 1
    if cntr > 200:
        break
fil.close()

Создадим функции, которые будут читать данные из файла и класть их в базу. Так как мы хотим повышенную производительность, класть будем по нескоклько тысяч документов за один раз.

In [60]:
def getCosycoLines(start, size):
    fil = open("noun_adj_inf_combs.txt")
    for i in range(start):
        _ = fil.readline()

    res=[]
    for i in range(size):
        r = fil.readline().split(";")
        #r={"noun":r[0], "adj":r[1], "frq": r[2]}
        r2 = { "index": {"_index": "cosyco", "_type": "combinations"} }
        res.append(r2)
        r2 = {"noun":r[0].strip(), "adj":r[1].strip()}
        res.append(r2)
        
    fil.close()
    return res


Ячейка ниже выполнялась 36 минут, хотя большая часть времени потрачена на формирование списка с сочетаниями. Попробуем по-другому.

In [61]:
rrr = getCosycoLines(0, 1000000)

In [54]:
%%time
#bulk(es, getCosycoLines(0, 1000000), index="cosyco", doc_type='combinations')
# Возвращает информацию об ошибках - кто был добавлен, а кто нет.
_ = es.bulk(rrr, index="cosyco", doc_type='combinations')


TransportError: TransportError(413, '')

Код ниже занял около 5 секунд: 20000 записей в секунду. Не так много, но гораздо лучше, чем было.

In [62]:
%%time

for i in tqdm(range(100)):
    passed=False
    while not passed:
        try:
            acq = es.bulk(getCosycoLines(i*10000, 10000), index="cosyco", doc_type='combinations')
            passed=True
        except:
            print(acq)



CPU times: user 57.6 s, sys: 1.2 s, total: 58.7 s
Wall time: 1min 55s


Посмотрим как со скоростью поиска.

In [63]:
%%time
res = es.search(index="cosyco", body={"query": {"match_all": {}}})
res

CPU times: user 3.32 ms, sys: 0 ns, total: 3.32 ms
Wall time: 8.02 s


{'took': 7874,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 1.0,
  'hits': [{'_index': 'cosyco',
    '_type': 'combinations',
    '_id': 'yk4c3HUBQZpeSNrxcSTv',
    '_score': 1.0,
    '_source': {'noun': 'ЁЖИК', 'adj': 'АГРЕССИВНЫЙ', 'frq': '1\n'}},
   {'_index': 'cosyco',
    '_type': 'combinations',
    '_id': 'y04c3HUBQZpeSNrxcyQE',
    '_score': 1.0,
    '_source': {'noun': 'ЁЖИК', 'adj': 'АККУРАТНЫЙ', 'frq': '5\n'}},
   {'_index': 'cosyco',
    '_type': 'combinations',
    '_id': 'zE4c3HUBQZpeSNrxcyRM',
    '_score': 1.0,
    '_source': {'noun': 'ЁЖИК', 'adj': 'АНЕКДОТИЧНЫЙ', 'frq': '1\n'}},
   {'_index': 'cosyco',
    '_type': 'combinations',
    '_id': 'zU4c3HUBQZpeSNrxcySt',
    '_score': 1.0,
    '_source': {'noun': 'ЁЖИК', 'adj': 'АНОНИМНЫЙ', 'frq': '3\n'}},
   {'_index': 'cosyco',
    '_type': 'combinations',
    '_id': 'zk4c3HUBQZpeSNrxcyTv',
    '_score': 1.

In [64]:
%%time
res = es.search(index="cosyco", body={"query": {"match": {"adj":"БЕЛЫЙ"}}})
print(res)

{'took': 313, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 840, 'relation': 'eq'}, 'max_score': 7.0817156, 'hits': [{'_index': 'cosyco', '_type': 'combinations', '_id': '1E4c3HUBQZpeSNrxdiRf', '_score': 7.0817156, '_source': {'noun': 'ЁЖИК', 'adj': 'БЕЛЫЙ', 'frq': '4\n'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'nU4f3HUBQZpeSNrx3yU9', '_score': 7.0817156, '_source': {'noun': 'ЁЖИК', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'fU4f3HUBQZpeSNrx3ydE', '_score': 7.0817156, '_source': {'noun': 'ЁЛОЧКА', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': '604f3HUBQZpeSNrx3ydF', '_score': 7.0817156, '_source': {'noun': 'ЁМКОСТЬ', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'Pk4f3HUBQZpeSNrx3ypJ', '_score': 7.0817156, '_source': {'noun': 'ЁРШИК', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'i04f3HUBQZpeSNrx3y

10 миллисекунд при поиске в миллионе документов - очень неплохо.

Хотя если вытаскивать все документы... Для этого надо добавить поле `size`.

In [68]:
%%time
res = es.search(index="cosyco", body={"size":1000, "query": {"match": {"adj":"белый"}}})
print(res)

{'took': 55, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 840, 'relation': 'eq'}, 'max_score': 7.0817156, 'hits': [{'_index': 'cosyco', '_type': 'combinations', '_id': '1E4c3HUBQZpeSNrxdiRf', '_score': 7.0817156, '_source': {'noun': 'ЁЖИК', 'adj': 'БЕЛЫЙ', 'frq': '4\n'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'nU4f3HUBQZpeSNrx3yU9', '_score': 7.0817156, '_source': {'noun': 'ЁЖИК', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'fU4f3HUBQZpeSNrx3ydE', '_score': 7.0817156, '_source': {'noun': 'ЁЛОЧКА', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': '604f3HUBQZpeSNrx3ydF', '_score': 7.0817156, '_source': {'noun': 'ЁМКОСТЬ', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'Pk4f3HUBQZpeSNrx3ypJ', '_score': 7.0817156, '_source': {'noun': 'ЁРШИК', 'adj': 'БЕЛЫЙ'}}, {'_index': 'cosyco', '_type': 'combinations', '_id': 'i04f3HUBQZpeSNrx3yt

20-400 миллисекунд, время зависит от того, насколько БД подгрузила кэш и чем она занята еще.

Ну, неплохо.


In [69]:
len(res["hits"]["hits"])

840

---

Попробуем решить еще одну задачу. Возьмем Синтагрус из [Universal Dependencies](https://universaldependencies.org/). Загрузим из него пары слов, связанных синтаксическими отношениями. При этом будем различать расположение главного и зависимого слова - главное справа (->) или главное слева (<-). Далее организуем поиск по этим отношениям, показывая примеры интересующих нас отношений.

Для этого нам придется завести два индекса. В первом будут хранится предложения, во втором - отношения между словами.

**NB**

Вроде бы в одном индексе можно хранить несколько типов документов. Но у меня почему-то не получитлось. Заодно не полчилось организовать хранение потомков и родителей как поддокументов в документе (точнее организовать по ним поиск). Поэтому решение несколько странное.

In [70]:
es=Elasticsearch()

Так как мы будем проводить аггрегацию по некоторым текстовым полям, то мы добавили к ним `"fielddata": True`.

In [77]:
es.indices.delete("syntagrus")
es.indices.delete("syntagrus_rel")

es.indices.create(index = "syntagrus")
mapit = {"sentences" : {"properties" : {"sentID" : {"type" : "integer"},
                                        "wordPos" : {"type" : "integer"},
                                        "token" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                        "lemma" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                        "PoS" : {"type" : "keyword"},
                                        "tag" : {"type" : "keyword"},
                                        "parent" : {"type" : "integer"},
                                        "conn" : {"type" : "text", "fielddata": True}}},
      
         
        }

es.indices.put_mapping(index = "syntagrus", 
                       doc_type = 'sentences', 
                       body = mapit,
                       include_type_name=True)

es.indices.create(index = "syntagrus_rel")
mapit = {"relations" : {"properties" : {"sentID" : {"type" : "integer"},
                                        "conn" : {"type" : "keyword"},
                                        "dir" : {"type" : "keyword"},
                                        "child_PoS" : {"type" : "keyword"},
                                        "parent_PoS" : {"type" : "keyword"},
                                        "parent":{"type": "object", "properties": {
                                            "wordPos" : {"type" : "integer"},
                                            "token" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                            "lemma" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                            "PoS" : {"type" : "keyword"},
                                            "tag" : {"type" : "keyword"}
                                        }},
                                        "child":{"type": "object", "properties": {
                                            "wordPos" : {"type" : "integer"},
                                            "token" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                            "lemma" : {"type" : "text", "analyzer" : "russian", "fielddata": True},
                                            "PoS" : {"type" : "keyword"},
                                            "tag" : {"type" : "keyword"}
                                        }}}
                       }}
es.indices.put_mapping(index = "syntagrus_rel", 
                       doc_type = 'relations', 
                       body = mapit,
                       include_type_name=True)

{'acknowledged': True}

In [75]:
!head -n 50 "/home/edward/projects/Alien_bases/Universal Dependencies/ud-treebanks-v2.4/UD_Russian-SynTagRus/ru_syntagrus-ud-dev.conllu"

# sent_id = 2013Algoritm.xml_1
# text = Алгоритм, от имени учёного аль-Хорезми, - точный набор инструкций, описывающих порядок действий исполнителя для достижения результата решения задачи за конечное время.
1	Алгоритм	алгоритм	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	12	nsubj	12:nsubj	SpaceAfter=No
2	,	,	PUNCT	_	_	4	punct	4:punct	_
3	от	от	ADP	_	_	4	case	4:case	_
4	имени	имя	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	1	conj	1:conj	_
5	учёного	ученый	NOUN	_	Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing	4	nmod	4:nmod	_
6	аль	аль	PART	_	_	5	appos	5:appos	SpaceAfter=No
7	-	-	PUNCT	_	_	8	punct	8:punct	SpaceAfter=No
8	Хорезми	Хорезми	PROPN	_	Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing	6	flat:name	6:flat:name	SpaceAfter=No
9	,	,	PUNCT	_	_	8	punct	8:punct	_
10	-	-	PUNCT	_	_	1	punct	1:punct	_
11	точный	точный	ADJ	_	Case=Nom|Degree=Pos|Gender=Masc|Number=Sing	12	amod	12:amod	_
12	набор	набор	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	0	root	0:root	_
13	инструкц

Читаем все тексты построчно.

In [78]:
#with open("/home/edward/projects/Alien_bases/Universal Dependencies/ud-treebanks-v2.3/UD_Russian-SynTagRus/ru_syntagrus-ud-train.conllu") as infile:
#with open("/home/edward/projects/universal dependencies/ud-treebanks-v2.3/UD_Russian-SynTagRus/ru_syntagrus-ud-test.conllu") as infile:
with open("/home/edward/projects/Alien_bases/Universal Dependencies/ud-treebanks-v2.3/UD_Russian-SynTagRus/ru_syntagrus-ud-dev.conllu") as infile:
    lines=infile.readlines()

Напишем несколько функций: выделяющую предложения из текстов и помещающую данные в индексы.

In [79]:
def readSentence(lines, pos):
    #print("pos=",pos, "line=", lines[pos])
    sent=[]
    while pos<len(lines) and lines[pos]!="\n":
        if lines[pos][0]!='\n' and lines[pos][0]!='#':
            sent.append(lines[pos][:-2].split("\t"))
        pos+=1
    pos+=1
    return sent, pos
    
def putSentence(es, sent, sent_no):
    data=[]
    rels=[]
    for word in sent:
        if '.' in word[0]:
            continue
        data.append({ "index": {"_index": "syntagrus", "_type": "sentences"} })
        data.append({"sentID" : sent_no,
              "wordPos" : word[0],
              "token" : word[1],
              "lemma" : word[2],
              "PoS" : word[3],
              "tag" : word[5],
              "parent" : word[6],
              "conn" : word[7]})
        rels.append({ "index": {"_index": "syntagrus_rel", "_type": "relations"} })
        rels.append({"sentID" : sent_no,
                     "conn" : word[7],
                     "dir" : '->' if word[0]<sent[int(word[6])][0] else '<-',
                     "child_PoS" : word[3],
                     "parent_PoS" : sent[int(word[6])][3],
                     "child":{"properties": {
                         "wordPos" : word[0],
                         "token" : word[1],
                         "lemma" : word[2],
                         "PoS" : word[3],
                         "tag" : word[5]
                     }},
                     "parent":{"properties": {
                         "wordPos" : sent[int(word[6])][0],
                         "token" : sent[int(word[6])][1],
                         "lemma" : sent[int(word[6])][2],
                         "PoS" : sent[int(word[6])][3],
                         "tag" : sent[int(word[6])][5]
                     }}}
        )
        
        #es.index(index="syntagrus", doc_type="sentences", body=data)
        
    es.bulk(rels, index="syntagrus_rel", doc_type='relations')        
    es.bulk(data, index="syntagrus", doc_type='sentences') 

Помещаем данные в индексы. Получается несколько предложений в секунду.

В примере ниже есть первая аггрегация. Чтобы ее задать используются ключевые слова `aggs` или `aggregations`. `"size": 0` необходимо, так как если его не задать, то помимо агрегированной информации будут выдаваться еще и документы, по которым проводилась агрегация.

In [80]:
res=es.search(index = "syntagrus", body = {"size": 0, "aggs": {"max_id": {"max": {"field": "sentID" }}}})
# Получение максимального номера предложения, уже хранимого в базе.
print(res)
sent_no=res["aggregations"]["max_id"]["value"]
sent_no=1 if sent_no==None else sent_no+1
sent_no=int(sent_no)

# Пошли по всему файлу. 
pos=0
while pos<len(lines):
    sent, pos=readSentence(lines, pos)
    #try:
    putSentence(es, sent, sent_no)
    #except:
    #    pass
    sent_no+=1
    print(sent_no, end="\r")

{'took': 878, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'max_id': {'value': None}}}


KeyboardInterrupt: 

Теперь посмотрим сколько есть "правых" и "левых" связей. Для этого зададим название поля для агрегации - dirs, поиск будет вестись по терминам - terms, поле поиска - "field" : "dir", сортировка по ключевому слову: "order": {"_term": "asc"}, выдать первые 100 ключевых слов: "size":"100" (по умолчанию - 10).

**NB**
У меня в параллель идет загрузка данных, так что числа будут несколько отличаться от запроса к запросу.

In [51]:
req={
    "size":0,
    "aggs" : {
        "dirs": {"terms" : { "field" : "dir", "size":"100", "order": {"_term": "asc"} }}
    }
}
es.search(index = "syntagrus_rel", body = req)

{'took': 64,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'dirs': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': '->', 'doc_count': 35778},
    {'key': '<-', 'doc_count': 32922}]}}}

Как видим, левого ветвления в русском языке в полтора раза больше. Но это ничего не значит. Посмотрим теперь зависимость от части речи главного и зависимого слов. Так как используется вложенная агрегация, то внутри одной мы вставляем другую: внутри агрегации "p_poses" будет агрегация "ch_poses", а в ней - "dirs".

In [31]:
req={
    "size":0,
    "aggs" : {
        "p_poses" : {
            "terms" : { "field" : "parent_PoS", "size":"100", "order": {"_term": "asc"}}, 
            "aggs" : {
                      "ch_poses" : {
                                    "terms" : { "field" : "child_PoS", "size":100, "order": {"_term": "asc"}}, 
                                    "aggs" : {
                                                "dirs": {"terms" : { "field" : "dir", "size":100, "order": {"_term": "asc"} }}
                                    }
                        }
                
            }
            
        }
    }
}
es.search(index = "syntagrus_rel", body = req)


{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'aggregations': {'p_poses': {'buckets': [{'ch_poses': {'buckets': [{'dirs': {'buckets': [{'doc_count': 151,
           'key': '->'},
          {'doc_count': 303, 'key': '<-'}],
         'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 0},
        'doc_count': 454,
        'key': 'ADJ'},
       {'dirs': {'buckets': [{'doc_count': 83, 'key': '->'},
          {'doc_count': 3, 'key': '<-'}],
         'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 0},
        'doc_count': 86,
        'key': 'ADP'},
       {'dirs': {'buckets': [{'doc_count': 454, 'key': '->'},
          {'doc_count': 53, 'key': '<-'}],
         'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 0},
        'doc_count': 507,
        'key': 'ADV'},
       {'dirs': {'buckets': [{'doc_count': 122, 'key': '->'},
          {'doc_count': 46, 'key': '<-'}],
         'doc_count_error_upper_bound': 0,
         'sum

Первые результаты показаны для прилагательного. И первый его потомок - другое прилагательное. Это немного странно. Посмотрим что за слова друг другу подчиняются и в каком предложении.

При помощи `must` мы говорим БД, что мы объединяем несколько полей, которые должны быть в документе.

In [82]:
req={"query": {"bool": {"must":[{"match": {"child_PoS": "NOUN"}}, {"match": {"parent_PoS": "ADJ"}}]}}}
    
es.search(index = "syntagrus_rel", body = req)

{'took': 40,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1967, 'relation': 'eq'},
  'max_score': 3.97433,
  'hits': [{'_index': 'syntagrus_rel',
    '_type': 'relations',
    '_id': 'rF023HUBQZpeSNrxYWgJ',
    '_score': 3.97433,
    '_source': {'sentID': 5,
     'conn': 'obl',
     'dir': '<-',
     'child_PoS': 'NOUN',
     'parent_PoS': 'ADJ',
     'child': {'properties': {'wordPos': '9',
       'token': 'завершения',
       'lemma': 'завершение',
       'PoS': 'NOUN',
       'tag': 'Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing'}},
     'parent': {'properties': {'wordPos': '7',
       'token': 'независимыми',
       'lemma': 'независимый',
       'PoS': 'ADJ',
       'tag': 'Case=Ins|Degree=Pos|Number=Plur'}}}},
   {'_index': 'syntagrus_rel',
    '_type': 'relations',
    '_id': 'rV023HUBQZpeSNrxZGmS',
    '_score': 3.97433,
    '_source': {'sentID': 10,
     'conn': 'nmod',
     'dir': '<-',
     'child_PoS

Для выведенных предложений посмотрим их пару и предложение, в которое они входят.

In [83]:
def printStrangeRelation(ch_PoS, p_PoS, sentID):
    req={"query": {"bool": {"must":[{"match": {"child_PoS": ch_PoS}}, {"match": {"parent_PoS": p_PoS}}, {"match": {"sentID":sentID}}]}}}
    res1=es.search(index = "syntagrus_rel", body = req)
    print(res1["hits"]["hits"][0]["_source"]["child"], res1["hits"]["hits"][0]["_source"]["dir"], res1["hits"]["hits"][0]["_source"]["parent"])
    res=es.search(index = "syntagrus", body = {"query":{"match": {"sentID":sentID}}, "sort":"wordPos", "size":1000})
    print([(r["_source"]["token"], r["_source"]["wordPos"]) for r in res["hits"]["hits"]])

In [84]:
printStrangeRelation('ADJ', 'ADJ', 5)

{'properties': {'wordPos': '7', 'token': 'независимыми', 'lemma': 'независимый', 'PoS': 'ADJ', 'tag': 'Case=Ins|Degree=Pos|Number=Plur'}} <- {'properties': {'wordPos': '7', 'token': 'независимыми', 'lemma': 'независимый', 'PoS': 'ADJ', 'tag': 'Case=Ins|Degree=Pos|Number=Plur'}}
[('Независимые', '1'), ('инструкции', '2'), ('или', '3'), ('инструкции', '4'), (',', '5'), ('ставшие', '6'), ('независимыми', '7'), ('из-за', '8'), ('завершения', '9'), ('работы', '10'), ('инструкций', '11'), (',', '12'), ('от', '13'), ('которых', '14'), ('они', '15'), ('зависят', '16'), (',', '17'), ('могут', '18'), ('выполняться', '19'), ('в', '20'), ('произвольном', '21'), ('порядке', '22'), (',', '23'), ('параллельно', '24'), ('или', '25'), ('одновременно', '26'), (',', '27'), ('если', '28'), ('это', '29'), ('позволяют', '30'), ('используемые', '31'), ('процессор', '32'), ('и', '33'), ('операционная', '34'), ('система', '35'), ('.', '36')]


Интересная логика, но почему нет.